In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from tensorflow.keras.datasets import  cifar10

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [ ]:
class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse",	"ship",	"truck"]

# Mobilenet V2

In [ ]:
Img_size = 224
Batch_size = 64

In [ ]:
# One-hot encode labels
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

# Define resize size (e.g. 224 for MobileNetV2)
IMG_SIZE = 224
BATCH_SIZE = 16  # adjust based on your GPU RAM

# Create tf.data datasets
X_train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train_cat))
X_test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test_cat))

# Preprocessing function
def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))  # resize to (224,224)
    image = preprocess_input(image)                       # MobileNetV2 preprocess
    return image, label

# Apply preprocessing + batching + prefetching
X_train_ds = X_train_ds.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
X_test_ds  = X_test_ds.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [ ]:
for images, labels in X_train_ds.take(1):
    print(images.shape)
    print(labels.shape)

In [ ]:
import matplotlib.pyplot as plt

for images, labels in X_train_ds.take(1):
    plt.imshow((images[0].numpy() + 1) / 2)  # MobileNet preprocess_input scales pixels
    plt.show()
    break

In [ ]:
# Build transfer learning model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base

In [ ]:
model_adam = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(), # Pool spatial features into a vector
    layers.Dense(128, activation='relu'), # Fully connected layer to learn new patterns
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')

])

In [ ]:
model_adam.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
history_adam_mobnet = model_adam.fit(X_train_ds,validation_data=X_test_ds, epochs=10, callbacks= ModelCheckpoint(filepath='mobilenet_adam_bm.keras',monitor='val_loss',
    save_best_only=True,mode='min',verbose=1))

In [ ]:
model_2 = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(), # Pool spatial features into a vector
    layers.Dense(128, activation='relu'), # Fully connected layer to learn new patterns
    layers.Dense(10, activation='softmax')
])

In [ ]:
model_2.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history_sgd_mobilenet = model_2.fit(X_train_ds,validation_data=X_test_ds, epochs=10, callbacks= ModelCheckpoint(filepath='mobilenet_sgd_bm.keras',monitor='val_loss',
    save_best_only=True,mode='min',verbose=1))

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
load_model_a = load_model('mobilenet_adam_bm.keras')
load_model_s = load_model('mobilenet_sgd_bm.keras')

In [ ]:
est_loss_a, test_acc_a = load_model_a.evaluate(X_test_ds)
print(f"Test Accuracy: {test_acc_a:.4f}")
print(f"Test Loss: {est_loss_a:.4f}")

In [ ]:
est_loss_s, test_acc_s = load_model_s.evaluate(X_test_ds)
print(f"Test Accuracy: {test_acc_s:.4f}")
print(f"Test Loss: {est_loss_s:.4f}")

In [ ]:
# Predictions
y_pred_probs_a = load_model_a.predict(X_test_ds)
y_pred_a = np.argmax(y_pred_probs_a, axis=1)

In [ ]:
# Predictions
y_pred_probs_s = load_model_s.predict(X_test_ds)
y_pred_s = np.argmax(y_pred_probs_s, axis=1)

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(y_test, y_pred_a, target_names=class_names)
print(report)

In [ ]:
report = classification_report(y_test, y_pred_s, target_names=class_names)
print(report)

In [ ]:
cm = confusion_matrix(y_test, y_pred_a)

plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()


In [ ]:
cm = confusion_matrix(y_test, y_pred_s)

plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds',
            xticklabels=class_names,
            yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()


In [ ]:
base_model.trainable = True

# Freeze all layers except the last N (e.g., last 20 layers)
fine_tune_at = len(base_model.layers) - 20
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [ ]:
from tensorflow.keras import optimizers

## Fine tuning Mobile net

In [ ]:
load_model_a.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Continue training (fine-tuning)
fine_tune_epochs = 12
total_epochs = 3 + fine_tune_epochs  # if trained earlier 3 epochs

history_fine_a_mobnet = load_model_a.fit(
    X_train_ds,
    validation_data=X_test_ds,
    epochs=total_epochs,
    initial_epoch=3,callbacks= ModelCheckpoint(filepath='mobnetfintun_a_bm.keras',monitor='val_loss',
    save_best_only=True,mode='min',verbose=1))


In [ ]:
load_model_s.compile(
    optimizer=optimizers.SGD(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Continue training (fine-tuning)
fine_tune_epochs = 12
total_epochs = 6 + fine_tune_epochs  # if trained earlier 3 epochs

history_fine_s_mobnet = load_model_s.fit(
    X_train_ds,
    validation_data=X_test_ds,
    epochs=total_epochs,
    initial_epoch=3,callbacks= ModelCheckpoint(filepath='mobnetfintun_s_bm.keras',monitor='val_loss',
    save_best_only=True,mode='min',verbose=1))

In [ ]:
load_a_bm = load_model('mobnetfintun_a_bm.keras')
load_s_bm = load_model('mobnetfintun_s_bm.keras')

In [ ]:
test_loss_fntun, test_acc_fntun = load_a_bm.evaluate(X_test_ds)
print(f"Test Accuracy after fine-tuning: {test_acc_fntun:.4f}")
print(f"Test Loss after fine-tuning: {test_loss_fntun:.4f}")

In [ ]:
test_loss_fntun, test_acc_fntun = load_s_bm.evaluate(X_test_ds)
print(f"Test Accuracy after fine-tuning: {test_acc_fntun:.4f}")
print(f"Test Loss after fine-tuning: {test_loss_fntun:.4f}")

In [ ]:
# Unfreeze some layers for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-15]:  # keep most layers frozen, fine-tune last 30
    layer.trainable = False

load_a_bm.compile(
    optimizer=Adam(learning_rate=1e-5),  # lower LR for fine-tuning
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history_ft_final_a = load_a_bm.fit(
    X_train_ds,
    validation_data=X_test_ds,
    epochs=10,
    # batch_size=32
)